# Hate Speech

In [18]:
import numpy as np, pandas as pd
import os, sys
import math
import shutil
import zipfile
import string
import random
import json

# Paths and Variables

In [19]:
dataset_name = 'hate_speech'

In [20]:
input_dir = './raw'
output_dir = './processed'

outp_fname = os.path.join(output_dir, f'{dataset_name}.csv')
outp_train_fname = os.path.join(output_dir, f'{dataset_name}_train.csv')
outp_test_fname = os.path.join(output_dir, f'{dataset_name}_test.csv')
outp_test_key_fname = os.path.join(output_dir, f'{dataset_name}_test_key.csv')
outp_infer_instances = os.path.join(output_dir, f'{dataset_name}_infer_req.json')

In [21]:
inp_fname = 'labeled_data.csv'

# Read data into a DataFrame

In [22]:
data = pd.read_csv(os.path.join(input_dir, inp_fname))
print(data.shape)
data.head()

(24783, 7)


,id,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


# Prepare Data

In [23]:
id_col = "id"
target_col = "class"
text_col = "tweet"

In [24]:
data[target_col].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [25]:
data[target_col] =data[target_col].map({0: 'hate_speech', 1: 'offensive_language',2: 'neither'})

In [26]:
data[target_col].value_counts()

offensive_language    19190
neither                4163
hate_speech            1430
Name: class, dtype: int64

In [27]:
data = data[[id_col, target_col, text_col]]
data.head()

,id,class,tweet
0,0,neither,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,offensive_language,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,offensive_language,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,offensive_language,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,offensive_language,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


# Shuffle Data

In [28]:
# shuffle data
data = data.sample(frac=1, random_state=42)
print(data.shape)
data.head()

(24783, 3)


,id,class,tweet
2281,2326,offensive_language,934 8616\r\ni got a missed call from yo bitch
15914,16283,offensive_language,RT @KINGTUNCHI_: Fucking with a bad bitch you ...
18943,19362,neither,RT @eanahS__: @1inkkofrosess lol my credit ain...
16407,16780,offensive_language,RT @Maxin_Betha Wipe the cum out of them faggo...
13326,13654,offensive_language,Niggas cheat on they bitch and don't expect no...


# Insert Id Column

In [29]:
# insert Id column 
if id_col not in data.columns:
    N = data.shape[0]
    data.insert(0, id_col, np.arange(N))
    print(data.head())

# Utility to Save DF as a zipped file

In [30]:
def save_df_to_zipped_csv(df, ftype=None): 
    if ftype is not None: 
        suffix = f'_{ftype}'
    else: 
        suffix = ''
        
    zipped_f_name = f'{dataset_name}{suffix}.zip'
    archive_f_name = f'{dataset_name}{suffix}.csv'   
    compression_opts = dict(method='zip',
                        archive_name=archive_f_name)      
    df.to_csv(os.path.join(output_dir, zipped_f_name), index=False, compression=compression_opts )

# Save Main Data File

In [31]:
# # save original file as csv
# data.to_csv(outp_fname, index=False)

# save as zipped file 
save_df_to_zipped_csv(data)

# Train Test Split

In [32]:
from sklearn.model_selection import train_test_split
test_size = 0.1

data_train, data_test = train_test_split(data, test_size=test_size, random_state=42)


data_test_key = data_test[[id_col, target_col]].copy()
data_test = data_test.drop(columns=[target_col])
print(data_train.shape, data_test.shape, data_test_key.shape)

# # Save original files as csv
# data_train.to_csv(outp_train_fname, index=False)
# data_test.to_csv(outp_test_fname, index=False)
# data_test_key.to_csv(outp_test_key_fname, index=False)

(22304, 3) (2479, 2) (2479, 2)


In [33]:
# zip files
save_df_to_zipped_csv(data_train, "train")
save_df_to_zipped_csv(data_test, "test")
save_df_to_zipped_csv(data_test_key, "test_key")

# JSON inference request instance

In [34]:
instance = data_test.reset_index(drop=True).loc[0].to_dict()
infer_req_instance_dict = {  "instances": [ {**instance}, ] }
print(infer_req_instance_dict)
   
with open(outp_infer_instances, 'w', encoding='utf8') as f:
    json.dump(infer_req_instance_dict, f, indent=2, ensure_ascii=False)

{'instances': [{'id': 1561, 'tweet': '&#8220;@Son_Goku_Tho: Dr gero made android 18 to destroy da world das sum bitch nigga shit I would program a girl android to gimme sum neck 24/7&#8221;'}]}


In [35]:
# sorry about the deeply offensive tweets in the data!!!